In [ ]:
import os
import sys
import json
import torch
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm

In [ ]:
sys.path.insert(0, '../')
import utils.misc as workspace
from SkelPointNet import SkelPointNet 
from DataUtil import PCDataset, EllipsoidPcDataset
import FileRW as rw
import DistFunc as DF
import EvalUtil

In [ ]:
EXP_NAME = "abexp_16"
experiment_dir = os.path.join("../experiments/", EXP_NAME)
split_file = 'val_split.txt'
checkpoint = 'latest'

with open(os.path.join(experiment_dir, "specs.json"), "r") as f:
    specs = json.load(f)

In [ ]:
data_dir = specs["DataSource"]
point_num = specs["InputPointNum"]
skelpoint_num = specs["SkelPointNum"]
to_normalize = specs["Normalize"]

flag_sup = True  # supervision
flags_vec = specs.get("FlagsVec", [])
if not flags_vec:
    flags_vec = [0] * 4
flags_boolvec = [x==1 for x in flags_vec]
flag_spread, flag_radius, flag_medial, flag_spoke = flags_boolvec
print(f"FlagVec: {flags_vec}")
print(f"FLAGS: {flag_spread}, {flag_radius}, {flag_medial}, {flag_spoke}!!!")

# Assume Training/Test split file (given as cmd line arg) will be present in the experiment dir
pc_list_file = os.path.join(experiment_dir, split_file)
gpu = "0"

model_skel = SkelPointNet(
    num_skel_points=skelpoint_num,
    input_channels=0,
    use_xyz=True,
    flag_supervision=flag_sup,
    flag_spread=flag_spread,
    flag_radius=flag_radius,
    flag_medial=flag_medial,
    flag_spoke=flag_spoke
)

if torch.cuda.is_available():
    os.environ["CUDA_VISIBLE_DEVICES"] = gpu
    print("GPU Number:", torch.cuda.device_count(), "GPUs!")
    model_skel.cuda()
    model_skel.eval()

# Load the saved model
model_epoch = workspace.load_model_checkpoint(
    experiment_dir, checkpoint, model_skel
)
print(f"Evaluating model on using checkpoint={checkpoint} and epoch={model_epoch}.")

In [ ]:
# load data and evaluate
pc_list = rw.load_data_id(pc_list_file)
label_list = rw.load_label_id(pc_list_file)
label_dir = "../data/ellipsoid_data_5000/pointcloud_srep/"

eval_data = EllipsoidPcDataset(
    pc_list, label_list, 
    data_dir, label_dir,
    point_num, to_normalize
    )

In [ ]:
EvalUtil.test_results(experiment_dir, eval_data, model_skel, save_results=True, srep_res=False)